In [4]:
#senza overlap
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
 # importo la funzione per leggere il file
import warnings
from matplotlib.backends.backend_pdf import PdfPages
import os
import sys
import gdown

# ignoro gli avvisi durante l'esecuzione del codice -> quando faccio log10
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Percorso alla tua cartella locale (senza Drive mount)
drive_path = './'

# Scarica i file necessari da GitHub nel percorso locale
!wget -O "Lettura_Dati.py" https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Lettura_Dati.py
!wget -O "background_piuframe.py" https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/background_piuframe.py


# Assicurati che i file siano stati scaricati correttamente
assert os.path.exists('Lettura_Dati.py'), "Lettura_Dati.py non trovato!"
assert os.path.exists('background_piuframe.py'), "background_piuframe.py non trovato!"

# Importa i moduli dal percorso attuale
sys.path.insert(0, drive_path)
from Lettura_Dati import leggi_file
from background_piuframe import Background_PiuFrame

# Link con l'ID del file
file_url = "https://drive.google.com/uc?id=15LSYaKYgg9GrVJgwSE_dBTPDefqShAaD"
# Scarica il file da Google Drive
gdown.download(file_url, 'dati.txt', quiet=False)

# Leggi il file
file_path = "dati.txt"
frames = leggi_file(file_path)
frames_array = np.array(frames)

# Parametri
Nm = 9  # n di frame per blocco
T = 1 * Nm
N = 256 *Nm
bg = Background_PiuFrame(Nm)
background = np.array(bg)

# numero totale di finestre
total_windows = len(frames_array) // Nm

# inizializzo l'idx del blocco
block_idx = 0

# inizializzo il n di intervallo di tempo
intervallo_tempo = 1

max_amplitudes_BRPM = []  # lista per memorizzare i respiri
max_amplitudes_BPM = []  # lista per memorizzare il battito

# ciclo finché non raggiungo il numero totale di finestre
# uso un PdfPages per salvare tutti i plot in un unico PDF
pdf_filename = "grafici_outputM3.pdf"
with PdfPages(pdf_filename) as pdf:
  while intervallo_tempo <= total_windows:
      # idx inizio e fine del blocco
      start_idx = block_idx
      end_idx = block_idx + Nm

      block_frames = frames_array[start_idx:end_idx]
      block_frames = block_frames.flatten()
      # combino nm frames
      combined_fft = np.fft.fft(block_frames, n=N)
      # tolgo il background
      combined_fft_no_background = combined_fft[:N//2] - background[:N//2]

      # prendo solo le frequenze comprese tra 0 e 30 Hz
      xf = fftfreq(N, T / N)[:N//2]
      mask = (xf > 0) & (xf <= 30)

      # Filtra le frequenze tra 0 e 30 Hz
      xf_filtered = xf[mask]
      fft_amplitudes_filtered = np.abs(combined_fft_no_background[mask])

      # fft per respiro
      fft_amplitudes_selected = combined_fft_no_background[(xf > 0) & (xf <= 1)]
      max_amplitudes_BRPM_index = np.argmax(fft_amplitudes_selected)
      max_amplitudes_BRPM.append(np.abs(fft_amplitudes_selected[max_amplitudes_BRPM_index]))

      # fft per battito
      fft_amplitudes_selected2 = combined_fft_no_background[(xf >= 0.66) & (xf <= 6.66)]
      max_amplitudes_BPM_index = np.argmax(fft_amplitudes_selected2)
      max_amplitudes_BPM.append(np.abs(fft_amplitudes_selected2[max_amplitudes_BPM_index]))

      # Plot delle frequenze tra 0 e 30 Hz
      plt.figure(figsize=(8, 4))
      plt.plot(xf_filtered, fft_amplitudes_filtered)
      plt.title(f'Intervallo di tempo {intervallo_tempo} ({start_idx} - {end_idx})', fontsize=16)
      plt.xlabel('Frequenza [Hz]', fontsize=16)
      plt.ylabel('Ampiezza [au]', fontsize=16)
      plt.grid(True)

      # salvo il plot nel PDF
      pdf.savefig()
      plt.close()  # chiudo la figura per evitare sovrapposizioni

      block_idx += Nm
      intervallo_tempo += 1

# Calcola la media dei massimi valori di ampiezza
print("Massimi breath rate: ", max_amplitudes_BRPM)
BRPM = np.mean(max_amplitudes_BRPM)
print("Media dei massimi valori di ampiezza tra 0 e 1 Hz:", BRPM)
print("brpm", (BRPM / 2) * 60)

print("Massimi heartbeat: ", max_amplitudes_BPM)
BPM = np.mean(max_amplitudes_BPM)
print("Media dei massimi valori di ampiezza tra 0.66 e 6.66 Hz:", BPM)
print("bpm", (BPM / 2) * 60)

# salvo i risultati in un file di testo
output_filename = "risultatiM3.txt"
with open(output_filename, "w") as f:
    f.write(f"Media dei massimi valori di ampiezza tra 0 e 1 Hz: {BRPM}\n")
    f.write(f"BRPM: {(BRPM / 2) * 60}\n")
    f.write(f"Media dei massimi valori di ampiezza tra 0.66 e 6.66 Hz: {BPM}\n")
    f.write(f"BPM: {BPM * 60}\n")

print("File dei risultati e PDF dei grafici generati con successo!")

import shutil
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#percorso del drive condiviso e delle sottocartelle
percorso_su_drive = '/content/drive/Shareddrives/Radar/Vital parameter monitoring/Sense2GoL_pulse_data/FileOutput/'

# creo la directory se non esiste
os.makedirs(percorso_su_drive, exist_ok=True)

# sposto i file su Google Drive
shutil.move("grafici_outputM3.pdf", os.path.join(percorso_su_drive, "grafici_outputM3.pdf"))
shutil.move("risultatiM3.txt", os.path.join(percorso_su_drive, "risultatiM3.txt"))

print("File caricati con successo su Google Drive nel drive condiviso Radar!")


--2024-11-08 09:07:40--  https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Lettura_Dati.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2371 (2.3K) [text/plain]
Saving to: ‘Lettura_Dati.py’

Lettura_Dati.py     100%[===================>]   2.32K  --.-KB/s    in 0s      

2024-11-08 09:07:40 (33.4 MB/s) - ‘Lettura_Dati.py’ saved [2371/2371]

--2024-11-08 09:07:40--  https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/background_piuframe.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.gi

Downloading...
From: https://drive.google.com/uc?id=15LSYaKYgg9GrVJgwSE_dBTPDefqShAaD
To: /content/dati.txt
100%|██████████| 771k/771k [00:00<00:00, 91.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oF0mh5XYgx06jyDgADqXDVMGyvdEGKwO
To: /content/dati.txt
100%|██████████| 700k/700k [00:00<00:00, 97.6MB/s]


Massimi breath rate:  [1.4794022119372527, 5.911647564499196, 1.5061496023153098, 1.046408306075209, 1.1498564143897902, 2.9384190171387696, 3.712661110716268]
Media dei massimi valori di ampiezza tra 0 e 1 Hz: 2.534934889581685
brpm 76.04804668745055
Massimi heartbeat:  [2.304187569820425, 5.492102444371049, 4.0784158082828865, 2.043564922106633, 1.5936034534409806, 3.137935079002089, 3.811279011335296]
Media dei massimi valori di ampiezza tra 0.66 e 6.66 Hz: 3.208726898337051
bpm 96.26180695011153
File dei risultati e PDF dei grafici generati con successo!
Mounted at /content/drive
File caricati con successo su Google Drive nel drive condiviso Radar!


In [5]:
#con overlap
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from Lettura_Dati import leggi_file
import warnings
from matplotlib.backends.backend_pdf import PdfPages
import os
import gdown
import shutil
from google.colab import drive

# Ignoro gli avvisi durante l'esecuzione del codice
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Percorso alla tua cartella locale
drive_path = './'

# Scarica i file necessari da GitHub
!wget -O "Lettura_Dati.py" https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Lettura_Dati.py
!wget -O "background_piuframe.py" https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/background_piuframe.py

# Assicurati che i file siano stati scaricati correttamente
assert os.path.exists('Lettura_Dati.py'), "Lettura_Dati.py non trovato!"
assert os.path.exists('background_piuframe.py'), "background_piuframe.py non trovato!"

# Importa i moduli dal percorso attuale
from background_piuframe import Background_PiuFrame

# Scarica il file da Google Drive
file_url = "https://drive.google.com/uc?id=15LSYaKYgg9GrVJgwSE_dBTPDefqShAaD"
gdown.download(file_url, 'dati.txt', quiet=False)

# Leggi il file
file_path = "dati.txt"
frames = leggi_file(file_path)
frames_array = np.array(frames)

# Parametri
Nm = 9          # n di frame per blocco
overlap = 4     # sovrapposizione tra i blocchi
T = 1 * Nm
N = 256 * Nm
bg = Background_PiuFrame(Nm)
background = np.array(bg)
step = Nm - overlap

# Variabili per memorizzare i massimi
max_amplitudes_BRPM = []  # lista per memorizzare i respiri
max_amplitudes_BPM = []   # lista per memorizzare il battito

# PDF per salvare i grafici
pdf_filename = "grafici_outputM3_overlap.pdf"
with PdfPages(pdf_filename) as pdf:
    block_idx = 0
    intervallo_tempo = 1

    # Ciclo con sovrapposizione
    while block_idx + Nm <= len(frames_array):
        # Indici di inizio e fine del blocco
        start_idx = block_idx
        end_idx = block_idx + Nm

        block_frames = frames_array[start_idx:end_idx]
        block_frames = block_frames.flatten()
        combined_fft = np.fft.fft(block_frames, n=N)

        # Rimozione del background
        combined_fft_no_background = combined_fft[:N//2] - background[:N//2]

        # Frequenze e ampiezze filtrate
        xf = fftfreq(N, T / N)[:N//2]
        mask = (xf > 0) & (xf <= 30)
        xf_filtered = xf[mask]
        fft_amplitudes_filtered = np.abs(combined_fft_no_background[mask])

        # Calcolo BRPM (respiro)
        fft_amplitudes_selected = combined_fft_no_background[(xf > 0) & (xf <= 1)]
        max_amplitudes_BRPM_index = np.argmax(fft_amplitudes_selected)
        max_amplitudes_BRPM.append(np.abs(fft_amplitudes_selected[max_amplitudes_BRPM_index]))

        # Calcolo BPM (battito)
        fft_amplitudes_selected2 = combined_fft_no_background[(xf >= 0.66) & (xf <= 6.66)]
        max_amplitudes_BPM_index = np.argmax(fft_amplitudes_selected2)
        max_amplitudes_BPM.append(np.abs(fft_amplitudes_selected2[max_amplitudes_BPM_index]))

        # Plot delle frequenze tra 0 e 30 Hz
        plt.figure(figsize=(8, 4))
        plt.plot(xf_filtered, fft_amplitudes_filtered)
        plt.title(f'Intervallo di tempo {intervallo_tempo} ({start_idx} - {end_idx})', fontsize=16)
        plt.xlabel('Frequenza [Hz]', fontsize=16)
        plt.ylabel('Ampiezza [au]', fontsize=16)
        plt.grid(True)

        # Salva il grafico nel PDF
        pdf.savefig()
        plt.close()

        # Avanza al prossimo blocco con sovrapposizione
        block_idx += step
        intervallo_tempo += 1

# Calcolo della media dei massimi
BRPM = np.mean(max_amplitudes_BRPM)
BPM = np.mean(max_amplitudes_BPM)

# Salvataggio dei risultati
output_filename = "risultatiM3_overlap.txt"
with open(output_filename, "w") as f:
    f.write(f"Media dei massimi valori di ampiezza tra 0 e 1 Hz: {BRPM}\n")
    f.write(f"BRPM: {(BRPM / 2) * 60}\n")
    f.write(f"Media dei massimi valori di ampiezza tra 0.66 e 6.66 Hz: {BPM}\n")
    f.write(f"BPM: {(BPM / 2) * 60}\n")

print("Risultati calcolati e salvati con successo.")

# Caricamento su Google Drive
drive.mount('/content/drive', force_remount=True)
percorso_su_drive = '/content/drive/Shareddrives/Radar/Vital parameter monitoring/Sense2GoL_pulse_data/FileOutput/'
os.makedirs(percorso_su_drive, exist_ok=True)
shutil.move(pdf_filename, os.path.join(percorso_su_drive, pdf_filename))
shutil.move(output_filename, os.path.join(percorso_su_drive, output_filename))

print("File PDF e risultati caricati con successo su Google Drive nel drive condiviso Radar!")


--2024-11-08 09:14:26--  https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Lettura_Dati.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2371 (2.3K) [text/plain]
Saving to: ‘Lettura_Dati.py’

Lettura_Dati.py     100%[===================>]   2.32K  --.-KB/s    in 0s      

2024-11-08 09:14:26 (32.8 MB/s) - ‘Lettura_Dati.py’ saved [2371/2371]

--2024-11-08 09:14:26--  https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/background_piuframe.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.gi

Downloading...
From: https://drive.google.com/uc?id=15LSYaKYgg9GrVJgwSE_dBTPDefqShAaD
To: /content/dati.txt
100%|██████████| 771k/771k [00:00<00:00, 72.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oF0mh5XYgx06jyDgADqXDVMGyvdEGKwO
To: /content/dati.txt
100%|██████████| 700k/700k [00:00<00:00, 79.0MB/s]


Risultati calcolati e salvati con successo.
Mounted at /content/drive
File PDF e risultati caricati con successo su Google Drive nel drive condiviso Radar!
